In [1]:
import pandas as pd
import numpy as np
import torch

In [ ]:
data_path = 'data/'

# Prepare data

In [2]:
# all_data = pd.read_csv(data_path + 'label_clean_with_index.csv')
# impolite_data = all_data[all_data['label'] == 0]
# sample_impolite_data = pd.read_csv(data_path + 'sample_impolite.csv')
# non_sample_impolite_data = impolite_data[~impolite_data['Unnamed: 0'].isin(sample_impolite_data['Unnamed: 0'])]

In [3]:
# len(impolite_data), len(sample_impolite_data), len(non_sample_impolite_data)

(7923, 4000, 3923)

In [4]:
# non_sample_impolite_data.to_csv(data_path + 'non_sample_impolite.csv', index=False)

In [ ]:
non_sample_impolite_data = pd.read_csv(data_path + 'non_sample_impolite.csv')

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(non_sample_impolite_data['text'], test_size=0.2, random_state=42)

# DataLoader

In [ ]:
from torch.utils.data import Dataset, DataLoader

class ImpoliteDataset(Dataset):
    def __init__(self, X):
        self.X = X

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx]

In [ ]:
train_dataset = ImpoliteDataset(X_train)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = ImpoliteDataset(X_test)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# Model

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    LogitsProcessorList,
    MinLengthLogitsProcessor,
    TopKLogitsWarper,
    TemperatureLogitsWarper,
    StoppingCriteriaList,
    MaxLengthCriteria,
)

tokenizer = AutoTokenizer.from_pretrained("facebook/xglm-564M")
model = AutoModelForCausalLM.from_pretrained("facebook/xglm-564M")

In [ ]:
from torch import nn

class PrefixModel(nn.Module):
    def __init__(self, seq_len, prefix_len):
        super().__init__()
        self.prefix_len = prefix_len
        self.model = nn.Sequential([
            nn.Linear(seq_len, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, prefix_len),
        ])

    def forward(self, x):
        x = self.model(x)
        return x